In [1]:
import numpy as np
import pandas as pd

In [9]:
data = pd.read_csv('student.csv')
data = data.sample(frac=1)
X,Y = data.iloc[:,0:2].values,data.iloc[:,2].values
X.shape,Y.shape

((1000, 2), (1000,))

In [10]:
X = (X-X.mean(axis=0))/X.std(axis=0)
X[:5]

array([[-0.15188231,  0.28178058],
       [-0.28453062, -1.42473939],
       [ 0.77665584,  1.30569255],
       [ 0.51135923,  0.62308457],
       [-1.87631032, -1.15169619]])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=.2)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((800, 2), (200, 2), (800,), (200,))

#### Euclidean  
$ d = \sqrt{ \sum_{i=0}^{k} \big(X_i - Y_i \big)^2 } $

#### Manhatten
$ d = \sum_{i=0}^{k} \big| X_i - Y_i \big| $

#### MinKowski
$ d = \bigg( \sum_{i=0}^{k} \big( \big|X_i - Y_i\big| \big)^q  \bigg)^\frac{1}{q} $

In [13]:
def euclidean(X1,X2):
    return np.sqrt(np.sum(np.square(X1-X2),axis=1))

def manhatten(X1,X2):
    return np.sum(np.abs(X1-X2),axis=1)

def minkowski(X1,X2,q):
    return np.power(np.sum(np.power(np.abs(X1-X2),2),axis=1),1/q)

In [14]:
def predict(test,X,Y,k=5,distance='euclidean',q=4):
    """
        test: sample to test
        X : train data to find neighbors
        Y : target values
        k : number of neighbors to consider
        distance : choice of distance metric ( 'euclidean', 'manhatten', 'minkowski' ) 
        q : used for minkowski distance metric only
    """
    func = None
    if distance=='euclidean': func= euclidean
    elif distance=='manhatten': func=manhatten
    elif distance=='minkowski': func=lambda x,y: minkowski(x,y,q)
    else: print("Wrong Distance metric!!"); return None
    
    assert test.shape == (1,X.shape[1])
    y = func(X,test)
    ind = y.argsort()[:k]
    return np.mean(Y[ind])

In [26]:
predictions = []
for sample in X_test:
    sample = np.expand_dims(sample,axis=0) 
    predictions.append(predict(sample,X_train,Y_train,k=5,distance='euclidean'))
mae = np.mean(np.abs(predictions-Y_test))
mse = np.mean( np.square(predictions-Y_test) )
print(f"MAE: {mae:.4f}, MSE: {mse:.4f}")

MAE: 3.5940, MSE: 20.7664


In [27]:
predictions = []
for sample in X_test:
    sample = np.expand_dims(sample,axis=0) 
    predictions.append(predict(sample,X_train,Y_train,k=5,distance='manhatten'))
mae = np.mean(np.abs(predictions-Y_test))
mse = np.mean( np.square(predictions-Y_test) )
print(f"MAE: {mae:.4f}, MSE: {mse:.4f}")

MAE: 3.6260, MSE: 21.4016


In [34]:
predictions = []
for sample in X_test:
    sample = np.expand_dims(sample,axis=0) 
    predictions.append(predict(sample,X_train,Y_train,k=5,distance='minkowski',q=3))
mae = np.mean(np.abs(predictions-Y_test))
mse = np.mean( np.square(predictions-Y_test) )
print(f"MAE: {mae:.4f}, MSE: {mse:.4f}")

MAE: 3.5720, MSE: 20.4960
